In [1]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

trainPath = "C:\\Users\\ronit\\AI\\kaggle_data\\SkinCancerDataset(7c)\\train"

trainDG = ImageDataGenerator(rescale = 1./255,
                             rotation_range=90,
                             width_shift_range=0.3,
                             height_shift_range=0.3,
                             shear_range=0.2,
                             zoom_range=0.3,
                             horizontal_flip=True
                             )

trainG = trainDG.flow_from_directory(trainPath,
                                     target_size = (256, 256),
                                     class_mode = 'categorical',
                                     batch_size = 32,
                                     shuffle = True
                                     )

valPath = "C:\\Users\\ronit\\AI\\kaggle_data\\SkinCancerDataset(7c)\\validation"

valDG = ImageDataGenerator(rescale = 1./255,
                           rotation_range = 90,
                           width_shift_range=0.3,
                           height_shift_range=0.3,
                           shear_range = 0.2,
                           zoom_range = 0.3,
                           horizontal_flip = True
                           )
valG = valDG.flow_from_directory(valPath,
                                 target_size = (256, 256),
                                 class_mode = 'categorical',
                                 batch_size = 32,
                                 shuffle = True)


Found 7454 images belonging to 7 classes.
Found 1549 images belonging to 7 classes.


In [2]:
imgs, labels = next(trainG)
labels[:10]

array([[0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0.]], dtype=float32)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, AveragePooling2D, BatchNormalization

model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape = (256, 256, 3), data_format = "channels_last"))
model.add(BatchNormalization(axis = 1))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization(axis = 1))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization(axis = 1))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization(axis = 1))
model.add(MaxPool2D((2,2)))

model.add(Flatten())

model.add(Dropout(0.4))

model.add(Dense(128, activation='relu'))

model.add(Dense(7, activation='softmax'))

model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 254, 254, 64)      1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 254, 254, 64)      1016      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 125, 125, 128)     73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 125, 125, 128)     500       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 62, 62, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 60, 60, 256)      

In [5]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from datetime import datetime

estop = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=22)


log_dir = 'C:\\Users\\ronit\\AI\\kaggle_ml\\tensorboard'
timestamp = datetime.now().strftime("%Y-%m-%d--%H%M"+'_(skin-cancer-model7c2)')
log_dir = log_dir + '\\' + timestamp

tboard = TensorBoard(log_dir=log_dir, 
                    histogram_freq=1,
                    write_graph=True,
                    write_images=True,
                    update_freq='epoch',
                    profile_batch=2,
                    embeddings_freq=1)

cpoint = ModelCheckpoint(filepath = 'C:\\Users\\ronit\\AI\\kaggle_ml\\skin_cancer_model7c2_{val_accuracy: .3f}val_acc.h5', 
                         save_best_only = True, 
                         save_weights_only = False, 
                         monitor = 'val_accuracy',
                         mode = 'max')

In [6]:
history = model.fit(trainG, 
                    epochs=300, 
                    steps_per_epoch=116, 
                    validation_data = valG, 
                    verbose = 1, 
                    validation_steps=24,
                    callbacks = [estop, tboard, cpoint])

Epoch 1/300
116/116 [==============================] - 123s 1s/step - loss: 4.3344 - accuracy: 0.5862 - val_loss: 1.2385 - val_accuracy: 0.6615
Epoch 2/300
116/116 [==============================] - 106s 913ms/step - loss: 1.2479 - accuracy: 0.6259 - val_loss: 1.2173 - val_accuracy: 0.6315
Epoch 3/300
116/116 [==============================] - 94s 810ms/step - loss: 1.1321 - accuracy: 0.6472 - val_loss: 1.5594 - val_accuracy: 0.4948
Epoch 4/300
116/116 [==============================] - 94s 808ms/step - loss: 1.0775 - accuracy: 0.6526 - val_loss: 1.1318 - val_accuracy: 0.5911
Epoch 5/300
116/116 [==============================] - 95s 815ms/step - loss: 1.0619 - accuracy: 0.6589 - val_loss: 1.0172 - val_accuracy: 0.6628
Epoch 6/300
116/116 [==============================] - 96s 825ms/step - loss: 0.9952 - accuracy: 0.6624 - val_loss: 1.0523 - val_accuracy: 0.6120
Epoch 7/300
116/116 [==============================] - 93s 802ms/step - loss: 1.0141 - accuracy: 0.6546 - val_loss: 0.9623 - 

KeyboardInterrupt: 

In [7]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()

plt.show()

NameError: name 'history' is not defined